# 02-Build Image 

* Create GAR 
* generate dockerfile 
* build image from dockerfile on cloud build 



## Define Constants

In [2]:
PROJECT_ID = "demos-vertex-ai" 
REGION = "us-central1"

BUCKET_NAME="gs://" + PROJECT_ID + "-aip-pipeline-cicd"

PRIVATE_REPO = "automl-beans"


IMAGE_NAME = "automl-beans"  # @param {type:"string"}
IMAGE_TAG = "latest"  # @param {type:"string"}
IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}:{IMAGE_TAG}"

#### Create a private Docker repository

Your first step is to create your own Docker repository in Google Artifact Registry.

1.  Run the `gcloud artifacts repositories create` command to create a new Docker repository with your region with the description "docker repository".
    
2.  Run the `gcloud artifacts repositories list` command to verify that your repository was created.

In [3]:
! gcloud artifacts repositories create $PRIVATE_REPO --repository-format=docker \
    --location=$REGION \
    --description="Docker repository"

! gcloud artifacts repositories list

Create request issued for: [automl-beans]
Waiting for operation [projects/demos-vertex-ai/locations/us-central1/operation
s/1ef58fbe-df51-40f6-932d-f05724d47106] to complete...done.                    
Created repository [automl-beans].
Listing items under project demos-vertex-ai, across all locations.

                                                                    ARTIFACT_REGISTRY
REPOSITORY    FORMAT  MODE                 DESCRIPTION        LOCATION     LABELS  ENCRYPTION          CREATE_TIME          UPDATE_TIME          SIZE (MB)
asia.gcr.io   DOCKER  STANDARD_REPOSITORY                     asia                 Google-managed key  2022-10-18T19:20:32  2022-10-18T19:20:32  0
automl-beans  DOCKER  STANDARD_REPOSITORY  Docker repository  us-central1          Google-managed key  2022-10-19T20:07:45  2022-10-19T20:07:45  0
eu.gcr.io     DOCKER  STANDARD_REPOSITORY                     europe               Google-managed key  2022-10-18T19:20:25  2022-10-18T19:20:25  0
gcr.io       

#### Configure authentication to your private repo
Before you push or pull container images, configure Docker to use the gcloud command-line tool to authenticate requests to Artifact Registry for your region.

In [4]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
Docker configuration file updated.


In [5]:
%%writefile Dockerfile
# Download dependencies to /root/.local.
FROM python:3.7-slim AS builder
COPY requirements.txt .

RUN pip install -r requirements.txt

# # Pull new image to only keep the things we need.
# FROM python:3.7-slim
# WORKDIR /src

# Copy downloaded dependencies.
# COPY --from=builder /root/.local /root/.local
COPY . .

# Update python path.
# ENV PATH=/root/.local:$PATH

Overwriting Dockerfile


### Build the Docker container

Next, you build the Docker container image on Cloud Build -- the serverless CI/CD platform.

*Note:* Building the Docker container image may take 10 to 15 minutes.

In [7]:
! gcloud builds submit --region=$REGION --tag=$IMAGE_URI --timeout=1h ./

Creating temporary tarball archive of 14 file(s) totalling 120.9 KiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/jupyter/.config/gcloud/logs/2022.10.19/20.10.36.025444.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [./] to [gs://demos-vertex-ai_cloudbuild/source/1666210236.151484-de382cb4c2db40008bcefcb4fb371642.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/demos-vertex-ai/locations/us-central1/builds/38cbb593-407f-4e19-ab68-70eebf112130].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/38cbb593-407f-4e19-ab68-70eebf112130?project=746038361521 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "38cbb593-407f-4e19-ab68-70eebf112130"

FETCHSOURCE
Fetching storage object: gs://demos-vertex-ai_cloudbuild/source/1666210236.15